In [ ]:
Nettoyage du corpus :


In [ ]:
Créer un script Python,
pour extraire le texte d’un PDF,
nettoyer les en-têtes et pieds de page,
supprimer les sauts de page,
et conserver la structure des entrées concernant les sceaux, à savoir :

« Sceaux » ou « Sceaux impériaux » (sceaux royaux) — ce sont les titres de classification des données.

Chaque entrée suit généralement le format standard :

Numéro (ex. : 1, 2, 3…).

Nom du propriétaire et fonction officielle.

Datation (siècle ou période).

Description technique (diamètre, poids, état de conservation).

Transcription de l’inscription (en grec).

Commentaires détaillés (c’est-à-dire, comme mentionné précédemment, en citant de nombreuses références provenant des abréviations pour confirmer l’identité de la personne).

Exemple :
Sceaux impériaux
 1. Justinien Ier
Date : 527-565.
Inv. : 29.
Dia. : 19 mm.
Des. : état de conservation correct ; échancré aux orifices du canal ; flan rogné sur une bonne partie
du pourtour/frappe non centrée.
Inédit.
À l’avers, dans un cercle de feuillage stylisé, buste de Justinien Ier, de face, nimbé et
imberbe ; l’empereur est ceint d’un diadème orné d’une aigrette trifide et de pendilia ; il
porte une chlamyde retenue sur l’épaule droite par une fibule circulaire. Au pourtour,
inscription circulaire, dont il ne subsiste que la première moitié : dnivstini - ………
D(ominus) n(oster) Iustini[anus p(er)p(etuus) aug(ustus)].
692 AlESSIO SOPRACASA & VIVIen PrIgenT
Au revers, dans un cercle de grènetis, Victoire aux ailes légèrement éployées, avançant
de face, vêtue d’un chiton ; chacun des bras, dressé, tient une couronne qui ne semble
pas ornée de bandelettes. la Victoire est flanquée de chaque côté à mi-hauteur d’un
symbole. Celui de gauche semble une croix mais est abîmé ; à droite, il faut sans doute
voir un christogramme. Les bulles de Justin II présentent une césure après IVST, tandis que celles de Justin Ier
sont anépigraphes et offrent un portrait de l’empereur légèrement de trois-quarts, ce
qui permet d’attribuer notre sceau à Justinien 1. Les nombreux sceaux conservés de ce
dernier présentent généralement la Victoire tenant des couronnes auxquelles des rubans,
généralement trois, sont appendus. Tel n’est pas le cas ici. Si notre identification d’un
christogramme dans le champ à droite est correcte, cette pièce s’apparenterait par ailleurs
à une bulle de la collection de Dumbarton Oaks caractérisée par ce symbole de champ
de chaque côté de la Victoire.

Les pages 1-2 contiennent les informations éditoriales.

Du page 3 à 70, c’est le contenu principal ; ignorer les notes finales pour l’instant et analyser seulement ces pages.

Pages 71-79 : ABRÉVIATIONS, un tableau de correspondance des abréviations utilisées dans le domaine de la byzantinologie (revues et références).
Pages 80-93 : expliquent pourquoi l’auteur attribue ce sceau à cette personne, et la contribution de cette découverte à l’histoire byzantine.
Pages 94-96 : contiennent « l’identification » de ce PDF de 96 pages.

Même si le PDF que vous avez fait main contient seulement 96 pages, ces pages montrent le sommaire complet de l’ouvrage Mélanges Jean-Claude Cheynet.

Cet ouvrage compte plus de 900 pages et contient des dizaines de articles indépendants.
Le PDF dont vous disposez est probablement un tiré à part extrait du grand livre, commençant à la page 691 du volume : Sceaux byzantins de la collection Sopracasa.



# Extraction PDF — Sceaux byzantins de la collection Sopracasa

Ce notebook extrait le texte du PDF en :
- ne traitant que les **pages 3 à 70**
- supprimant les **en-têtes** (y < 70 pts)
- supprimant les **notes de bas de page / uniotes** (y > 540 pts)
- conservant les titres de section, titres d'entrée et champs structurés
- encodant en **UTF-8**

## Cellule 1 — Installation des dépendances

In [1]:
!pip install pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 61.8 MB/s eta 0:00:00


## Cellule 2 — Upload du PDF

Exécutez cette cellule pour uploader votre PDF directement dans Colab.

In [2]:
from google.colab import files

uploaded = files.upload()  # sélectionnez votre PDF
PDF_FILENAME = list(uploaded.keys())[0]
print(f"Fichier reçu : {PDF_FILENAME}")

Saving Sceaux_byzantins_de_la_collection_Soprac.pdf to Sceaux_byzantins_de_la_collection_Soprac (1).pdf
Fichier reçu : Sceaux_byzantins_de_la_collection_Soprac (1).pdf


## Cellule 3 — Paramètres (à ajuster si besoin)

In [3]:
# ── Fichiers ──────────────────────────────────────────────────────────────────
INPUT_PDF  = PDF_FILENAME          # chemin du PDF uploadé
OUTPUT_TXT = "sceaux_byzantins.txt" # nom du fichier de sortie

# ── Pages à extraire (index 0-based) ─────────────────────────────────────────
PAGE_START = 2    # page 3 du PDF  (index 2)
PAGE_END   = 69   # page 70 du PDF (index 69)

# ── Seuils de coordonnées verticales (en points, 1 pt = 1/72 pouce) ──────────
# Tout bloc dont le bord supérieur (y0) est :
#   < Y_HEADER_MAX  → en-tête    → supprimé
#   > Y_FOOTER_MIN  → note/pied  → supprimé
Y_HEADER_MAX = 70
Y_FOOTER_MIN = 540

# ── Champs structurés (toujours sur leur propre ligne) ───────────────────────
STRUCT_FIELDS = ("Date", "Inv.", "Dia.", "Des.")

print("Paramètres OK")

Paramètres OK


## Cellule 4 — Extraction et nettoyage

In [4]:
import re
import fitz  # PyMuPDF


def extract_page_blocks(page):
    """
    Retourne les blocs de texte de la page filtrés par zone :
    - exclut l'en-tête  (y0 < Y_HEADER_MAX)
    - exclut les notes  (y0 > Y_FOOTER_MIN)
    Triés par position verticale.
    """
    blocks = page.get_text("blocks")  # (x0, y0, x1, y1, text, ...)
    filtered = []
    for b in blocks:
        x0, y0, x1, y1, text, *_ = b
        text = text.strip()
        if not text:
            continue
        if y0 < Y_HEADER_MAX:   # en-tête
            continue
        if y0 > Y_FOOTER_MIN:   # notes de bas de page
            continue
        filtered.append((y0, text))
    filtered.sort(key=lambda t: t[0])
    return [text for _, text in filtered]


def format_structure(text):
    """Garantit que Date/Inv./Dia./Des. commencent sur leur propre ligne."""
    for label in STRUCT_FIELDS:
        text = re.sub(rf"(?<!\n)({re.escape(label)}\s*[\xa0:])", r"\n\1", text)
    return text


# ── Extraction ────────────────────────────────────────────────────────────────
doc = fitz.open(INPUT_PDF)
total_pages = doc.page_count
print(f"PDF ouvert : {total_pages} pages au total")

end = min(PAGE_END, total_pages - 1)
print(f"Traitement des pages {PAGE_START + 1} à {end + 1}")

all_parts = []
for idx in range(PAGE_START, end + 1):
    page = doc[idx]
    blocks_text = extract_page_blocks(page)
    if blocks_text:
        all_parts.append("\n\n".join(blocks_text))

doc.close()

# ── Nettoyage ─────────────────────────────────────────────────────────────────
full_text = "\n\n".join(all_parts)
full_text = format_structure(full_text)
full_text = full_text.replace("\xa0", " ")        # espaces insécables
full_text = re.sub(r"\n{3,}", "\n\n", full_text)  # lignes vides multiples
full_text = full_text.strip()

nb_lines = len(full_text.splitlines())
print(f"\nExtraction terminée : {len(full_text)} caractères | {nb_lines} lignes")

PDF ouvert : 96 pages au total
Traitement des pages 3 à 70

Extraction terminée : 186053 caractères | 2800 lignes


## Cellule 5 — Aperçu du résultat

In [5]:
# Affiche les 60 premières lignes pour vérification
print("\n".join(full_text.splitlines()[:60]))

SCEAUX BYZANTINS DE LA COLLECTION SOPRACASA

par Alessio Sopracasa & Vivien Prigent

La collection de bulles byzantines éditées ici a été assemblée après que la rencontre 
de son propriétaire avec le dédicataire de ce volume a éveillé son intérêt pour les petits 
monuments dont Jean-Claude Cheynet s’est fait une spécialité. Il nous a donc semblé 
particulièrement judicieux de lui en dédier l’édition dans ses mélanges en témoignage 
de reconnaissance pour son enseignement, lequel illustra souvent le fameux adage selon 
lequel « ce qui ne nous tue pas, nous rend plus fort ».

Sceaux impériaux

1. Justinien Ier

Date : 527-565.
Inv. : 29.
Dia. : 19 mm.
Des. : état de conservation correct ; échancré aux orifices du canal ; flan rogné sur une bonne partie 
du pourtour/frappe non centrée.
Inédit.

À l’avers, dans un cercle de feuillage stylisé, buste de Justinien Ier, de face, nimbé et 
imberbe ; l’empereur est ceint d’un diadème orné d’une aigrette trifide et de pendilia ; il 
porte une chl

## Cellule 6 — Sauvegarde et téléchargement

In [6]:
# Sauvegarde locale dans Colab
with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
    f.write(full_text)
print(f"Fichier sauvegardé : {OUTPUT_TXT}")

# Téléchargement automatique sur votre machine
files.download(OUTPUT_TXT)

Fichier sauvegardé : sceaux_byzantins.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## Notes de réglage

Si certaines notes ne sont pas filtrées ou si du contenu est manquant, ajustez dans la **Cellule 3** :

| Paramètre | Valeur par défaut | Effet |
|---|---|---|
| `Y_HEADER_MAX` | `70` | Augmenter → supprime plus de lignes en haut |
| `Y_FOOTER_MIN` | `540` | Diminuer → supprime plus de lignes en bas |
| `PAGE_START` | `2` (= page 3) | Première page à extraire |
| `PAGE_END` | `69` (= page 70) | Dernière page à extraire |

In [ ]:

Même si le fichier TXT a déjà été nettoyé des en-têtes et pieds de page du PDF, le contenu des notes de bas de page (par exemple : « 7. Seule la déformation… », « 8. Comme l'a fait remarquer… ») reste dans le corps du texte et possède exactement le même format que les numéros des entrées (par exemple : « 7. Nicétas, ostiaire… »).

Par conséquent, nous procéderons maintenant à un **second nettoyage**, tout en **extraitant les informations de manière structurée**.


In [ ]:
Analyse de données ET Extraction d’information :

In [ ]:
Tout d’abord, pour chaque sceau, ne conserver que :

Le titre du type (par exemple Sceaux impériaux).

Ensuite, extraire :

Le numéro + le nom (par exemple « 1. Justinien Ier »).

Les champs Date, Inv., Dia., Des., Inédit.

Le texte descriptif : sections « À l’avers… », « Au revers… » et les paragraphes de discussion académique.

# Parsing — Sceaux byzantins de la collection Sopracasa

Ce notebook parse le `.txt` et produit un CSV structuré.

**Logique de détection des entrées** : un `N. Nom` est une vraie entrée de catalogue
seulement si l'une des 8 lignes suivantes commence par `Date :`. Cela exclut
automatiquement les numéros de notes de bas de page (`7. Seule la déformation…`, etc.)
sans aucune regex fragilisante.

In [14]:
# ── Cellule 1 : upload ────────────────────────────────────────────────────────
from google.colab import files

uploaded = files.upload()   # sélectionnez sceaux_byzantins.txt
TXT_FILE = list(uploaded.keys())[0]
print(f"Fichier : {TXT_FILE}")

Saving sceaux_byzantins.txt to sceaux_byzantins.txt
Fichier : sceaux_byzantins.txt


In [25]:
# ── Cellule 2 : lecture + détection des entrées ───────────────────────────────
import re

with open(TXT_FILE, encoding="utf-8") as f:
    lines = f.readlines()
stripped = [l.rstrip("\n") for l in lines]

KNOWN_SECTIONS = {
    "Sceaux impériaux",
    "L'administration centrale",
    "Administration provinciale",
    "Les dignitaires",
    "L'Église",
    "Sceaux patronymiques",
}

RE_ENTRY = re.compile(r"^(\d+)\.\s+(.+)$")
RE_DATE  = re.compile(r"^Date\s*[:\xa0]\s*(.+)$")
RE_INV   = re.compile(r"^Inv\.\s*[:\xa0]\s*(.+)$")
RE_DIA   = re.compile(r"^Dia\.\s*[:\xa0]\s*(.+)$")
RE_DES   = re.compile(r"^Des\.\s*[:\xa0]\s*(.+)$")

# Détection : un "N. Nom" est une vraie entrée si, avant de croiser
# un autre "N. Nom", on trouve une ligne commençant par "Date :"
entry_indices = set()
for i, line in enumerate(stripped):
    m = RE_ENTRY.match(line.strip())
    if not m:
        continue
    for j in range(i + 1, min(i + 9, len(stripped))):
        s = stripped[j].strip()
        if RE_DATE.match(s):
            entry_indices.add(i)
            break
        if j > i + 1 and RE_ENTRY.match(s):
            break

print(f"{len(lines)} lignes lues, {len(entry_indices)} entrées détectées")
for i in sorted(entry_indices):
    print(f"  l.{i+1:4d}  {stripped[i].strip()[:70]}")

2800 lignes lues, 40 entrées détectées
  l.  14  1. Justinien Ier
  l.  44  2. Constantin IV, Constant II et Anastasie, basileis des Romains
  l. 203  3. Georges, patrice, juge du Velum et grand économe des fondations pie
  l. 238  4. Grègoras (?), spatharocandidat impérial et topotèrètès de l’Arithmo
  l. 291  5. Michel, vestarque et économe de la Néa
  l. 318  6. Nicétas (?) Sarônitès, protovestès (?) et domestique des Excubites
  l. 364  7. Nicétas, ostiaire impérial, épi tou koitonos, pincerne de l’augoust
  l. 411  8. Théodore, anthypatos, patrice, protospathaire impérial et logothète
  l. 438  9. David Tzipourélès, protospathaire et stratège
  l. 472  10. Étienne, vestarque et duc d’Antioche
  l. 514  11. Jean (?) Radènos, anthypatos, patrice, protospathaire impérial et 
  l. 553  12. Jean, protoproèdre et duc d’Antioche
  l. 579  13. Justinien, hypatos et topotèrètès de l’Opsikion
  l. 610  14. Léon, vestès, patrice et juge des Bucellaires
  l. 703  15. Léon, protospathaire impé

In [41]:
# ── Cellule 3 : extraction complète ──────────────────────────────────────────
entries = []
current_section = "(non classé)"
current = None
body_buf = []
in_des = False

def flush():
    global current, body_buf
    if current is not None:
        current["corps"] = "\n".join(body_buf).strip()
        entries.append(current)
    current = None
    body_buf = []

for i, line in enumerate(stripped):
    s = line.strip()

    if s in KNOWN_SECTIONS:
        current_section = s
        continue

    if i in entry_indices:
        flush()
        m = RE_ENTRY.match(s)
        current = {
            "section": current_section,
            "numero" : int(m.group(1)),
            "nom"    : m.group(2).strip(),
            "date"   : "",
            "inv"    : "",
            "dia"    : "",
            "des"    : "",
            "corps"  : "",
        }
        in_des = False
        continue

    if current is None:
        continue

    if RE_DATE.match(s):
        current["date"] = RE_DATE.match(s).group(1).rstrip(".")
        in_des = False
    elif RE_INV.match(s):
        current["inv"] = RE_INV.match(s).group(1).rstrip(".")
        in_des = False
    elif RE_DIA.match(s):
        current["dia"] = RE_DIA.match(s).group(1).rstrip(".")
        in_des = False
    elif RE_DES.match(s):
        current["des"] = RE_DES.match(s).group(1)
        in_des = True
    elif in_des:
        if not s:
            in_des = False
        elif s.startswith("Inédit") or s.startswith("//") or s.startswith("À l'") or s.startswith("Au revers"):
            in_des = False
            body_buf.append(s)
        else:
            current["des"] += " " + s
    else:
        body_buf.append(s)

flush()
print(f"Total entrées extraites : {len(entries)}")

# ── Ajout manuel des entrées bis/ter ─────────────────────────────────────────
extra_entries = [
    {
        "section" : "Administration provinciale",
        "numero"  : "20bis",
        "nom"     : "Oursel (Roussel) de Bailleul, proèdre et stratopédarque",
        "date"    : "dernier tiers du xie siècle",
        "inv"     : "", "dia"    : "", "des"    : "", "corps"  : "",
    },
    {
        "section" : "Administration provinciale",
        "numero"  : "20ter",
        "nom"     : "Zacharie, vestarque et stratège de Séleucie",
        "date"    : "1060-1080",
        "inv"     : "", "dia"    : "", "des"    : "", "corps"  : "",
    },
    {
        "section" : "Les dignitaires",
        "numero"  : "25bis",
        "nom"     : "Nicéphore Comnène, sébaste et grand drongaire de la flotte",
        "date"    : "xiie siècle (début)",
        "inv"     : "", "dia"    : "", "des"    : "", "corps"  : "",
    },
]

def insert_after(entries, num, new_entry):
    for i, e in enumerate(entries):
        if str(e["numero"]) == str(num):
            entries.insert(i + 1, new_entry)
            return

insert_after(entries, 20,      extra_entries[0])
insert_after(entries, "20bis", extra_entries[1])
insert_after(entries, 25,      extra_entries[2])
print(f"Total après ajout bis/ter : {len(entries)}")

Total entrées extraites : 40
Total après ajout bis/ter : 43


In [42]:
import pandas as pd
df = pd.DataFrame(entries, columns=FIELDS)
display(df)

,section,numero,nom,date,inv,dia,des,corps
0,Sceaux impériaux,1,Justinien Ier,527-565,29,19 mm,état de conservation correct ; échancré aux or...,"Inédit.\n\nÀ l’avers, dans un cercle de feuill..."
1,Sceaux impériaux,2,"Constantin IV, Constant II et Anastasie, basil...",viie siècle (668-685),56,32 mm,bon état de conservation ; large flan rogné su...,"Inédit.\n// : à strictement parler aucun 3, ma..."
2,Sceaux impériaux,3,"Georges, patrice, juge du Velum et grand écono...",xie siècle (troisième quart),55,23 mm,assez bon état de conservation ; frappe légère...,"Inédit.\n// : Zacos 2, no 469.\n\nÀ l’avers, d..."
3,Sceaux impériaux,4,"Grègoras (?), spatharocandidat impérial et top...",ixe siècle (dernier quart),71,24 mm,"bulle brisée dans sa partie supérieure, relief...","Inédit.\n\n23. n. Oikonomidès, L’évolution de ..."
4,Sceaux impériaux,5,"Michel, vestarque et économe de la Néa",milieu du xie siècle,53,26 mm,flan rogné sur le pourtour ; abrasion du visag...,"Inédit.\n\nÀ l’avers, buste de saint Michel, a..."
5,Sceaux impériaux,6,"Nicétas (?) Sarônitès, protovestès (?) et dome...",xie siècle (troisième quart),20,23 mm,"état de conservation bon au droit, moyen au re...","Inédit.\n\nÀ l’avers, dans un cercle de grènet..."
6,Sceaux impériaux,7,"Nicétas, ostiaire impérial, épi tou koitonos, ...",xe siècle (deuxième moitié),1,25 mm,état de conservation correct au droit ; au rev...,"Inédit.\n\nÀ l’avers, croix patriarcale sur qu..."
7,Sceaux impériaux,8,"Théodore, anthypatos, patrice, protospathaire ...",ixe siècle (dernier tiers),43,25 mm,bon état de conservation ; flan rogné sur le p...,"Inédit.\n\nÀ l’avers, croix patriarcale simple..."
8,Administration provinciale,9,"David Tzipourélès, protospathaire et stratège",xie siècle,19,22 mm,très échancré à l’orifice supérieur du canal ;...,"Inédit.\n\nÀ l’avers, dans un cercle de grènet..."
9,Administration provinciale,10,"Étienne, vestarque et duc d’Antioche",xie siècle (milieu),50,22 mm,un peu moins d’un tiers du sceau est perdu ; f...,"Inédit.\n// : V. Laurent, La chronologie des g..."


En comparant avec le PDF, nous constatons :

L’administration centrale
 3. Georges, patrice, juge du Velum et grand économe des fondations pieuses

 Administration provinciale
 9. David Tzipourélès, protospathaire et stratège


20bis
Date : dernier tiers du xie
 siècle.
Éd. : Schlumberger, Deux chefs normands (cité n. 89), p. 296 (avec dessin) ; Id., Sigillographie
(cité n. 89), p. 660 (avec dessin).
// : Cheynet, Sceaux de la collection Khoury (cité n. 89), no
 20, p. 436-437.
À l’avers, buste de la Théotokos Blachernitissa orante, de face, au nimbe de grènetis,

20ter
Date : 1060-1080.
À l’avers, bordure de grènetis ; buste finement gravé de la Théotokos Blachernitissa,
flanqué du sigle habituel ̅‒θ̅υ̅, Μ(ήτη)ρ Θ(εο)ῦ.
Au revers, bordure de grènetis ; légende sur sept lignes d’une très belle gravure, précédée


25bis
Date : xiie
 siècle (début).
Inédit

In [43]:
import pandas as pd
df = pd.DataFrame(entries, columns=FIELDS)

# ── Correction manuelle des sections ─────────────────────────────────────────
corrections = {
    range(3, 9): "L'administration centrale",
}
for num_range, section in corrections.items():
    df.loc[df["numero"].isin(num_range), "section"] = section

display(df)

,section,numero,nom,date,inv,dia,des,corps
0,Sceaux impériaux,1,Justinien Ier,527-565,29,19 mm,état de conservation correct ; échancré aux or...,"Inédit.\n\nÀ l’avers, dans un cercle de feuill..."
1,Sceaux impériaux,2,"Constantin IV, Constant II et Anastasie, basil...",viie siècle (668-685),56,32 mm,bon état de conservation ; large flan rogné su...,"Inédit.\n// : à strictement parler aucun 3, ma..."
2,L'administration centrale,3,"Georges, patrice, juge du Velum et grand écono...",xie siècle (troisième quart),55,23 mm,assez bon état de conservation ; frappe légère...,"Inédit.\n// : Zacos 2, no 469.\n\nÀ l’avers, d..."
3,L'administration centrale,4,"Grègoras (?), spatharocandidat impérial et top...",ixe siècle (dernier quart),71,24 mm,"bulle brisée dans sa partie supérieure, relief...","Inédit.\n\n23. n. Oikonomidès, L’évolution de ..."
4,L'administration centrale,5,"Michel, vestarque et économe de la Néa",milieu du xie siècle,53,26 mm,flan rogné sur le pourtour ; abrasion du visag...,"Inédit.\n\nÀ l’avers, buste de saint Michel, a..."
5,L'administration centrale,6,"Nicétas (?) Sarônitès, protovestès (?) et dome...",xie siècle (troisième quart),20,23 mm,"état de conservation bon au droit, moyen au re...","Inédit.\n\nÀ l’avers, dans un cercle de grènet..."
6,L'administration centrale,7,"Nicétas, ostiaire impérial, épi tou koitonos, ...",xe siècle (deuxième moitié),1,25 mm,état de conservation correct au droit ; au rev...,"Inédit.\n\nÀ l’avers, croix patriarcale sur qu..."
7,L'administration centrale,8,"Théodore, anthypatos, patrice, protospathaire ...",ixe siècle (dernier tiers),43,25 mm,bon état de conservation ; flan rogné sur le p...,"Inédit.\n\nÀ l’avers, croix patriarcale simple..."
8,Administration provinciale,9,"David Tzipourélès, protospathaire et stratège",xie siècle,19,22 mm,très échancré à l’orifice supérieur du canal ;...,"Inédit.\n\nÀ l’avers, dans un cercle de grènet..."
9,Administration provinciale,10,"Étienne, vestarque et duc d’Antioche",xie siècle (milieu),50,22 mm,un peu moins d’un tiers du sceau est perdu ; f...,"Inédit.\n// : V. Laurent, La chronologie des g..."


In [44]:
# ── Cellule 5 : export CSV ────────────────────────────────────────────────────
# 写 df 而non entries，pour que les corrections de section soient incluses
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"CSV : {OUTPUT_CSV}  ({len(df)} lignes)")
files.download(OUTPUT_CSV)

CSV : sceaux_parsed.csv  (43 lignes)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
print(df[["numero", "nom", "date"]].to_string())

#Selon le format de l’article, « Date » fait référence à la période de fabrication du sceau lui-même, c’est-à-dire au moment où ce sceau en plomb a été frappé — généralement équivalent à la période d’exercice officiel du propriétaire du sceau, et non à l’ensemble de sa vie.

   numero                                                                                    nom                                       date
0       1                                                                          Justinien Ier                                    527-565
1       2                          Constantin IV, Constant II et Anastasie, basileis des Romains                      viie siècle (668-685)
2       3                Georges, patrice, juge du Velum et grand économe des fondations pieuses               xie siècle (troisième quart)
3       4               Grègoras (?), spatharocandidat impérial et topotèrètès de l’Arithmos (?)                 ixe siècle (dernier quart)
4       5                                                 Michel, vestarque et économe de la Néa                       milieu du xie siècle
5       6                     Nicétas (?) Sarônitès, protovestès (?) et domestique des Excubites               xie siècle (troisième quart)
6       7  Nicétas, 

En observant la sortie ci-dessus :

Dans les données, une seule personne apparaît en double : Oursel (Roussel) de Bailleul, correspondant aux entrées 20 et 20bis — c’est exactement ce que vous aviez ajouté manuellement, comme prévu.

Les 39 autres entrées concernent des personnes distinctes, sans doublon inattendu.

Réfléchissons maintenant aux liens (lien) → c’est-à-dire que la définition des liens détermine ce dont les nœuds (nœud) ont besoin.

Le champ nom est très riche : il contient déjà :

Nom des personnes : Justinien, Georges, Nicétas…

Fonctions ou titres : patrice, stratège, protospathaire…

Lieux : Antioche, Dyrrachion, Séleucie…

Concernant les noms religieux : pour ces 43 entrées, les informations sur les images religieuses se trouvent dans le corps du texte (corps) — Théotokos, saint Michel, saint Georges… — et non dans le champ nom. Cette extraction nécessitera donc un traitement séparé du corps de texte.

Le graphique comprend deux parties :

Nœuds (Nodes) : sceaux, personnes, titres, éléments religieux, etc.

Relations (Relations) : liens spatiaux ou historiques entre ces nœuds.

L’idée est de combiner texte, images et relations pour analyser le réseau d’information des sceaux.



Lien (edges) :

Figure 1 (Harry Potter) : réseau de cooccurrence de mots, Weight = nombre de cooccurrences.

Figure 2 : cooccurrence de nœuds avec identifiants numériques.

### edges.csv
| Source | Target | Weight | Type | relation |
|---|---|---|---|---|
| 1 | 2 | 1 | Non dirigé | personne_titre |
| 1 | 3 | 1 | Non dirigé | personne_lieu |
| 1 | 5 | 1 | Non dirigé | personne_siecle |
| 1 | 4 | 1 | Non dirigé | personne_religion |
| 4 | 5 | 3 | Non dirigé | religion_siecle |
| 2 | 5 | 2 | Non dirigé | titre_siecle |

- `Source` / `Target` : Id numériques correspondant aux nœuds
- `Weight` : fréquence de co-occurrence dans un même sceau
- `Type` : Non dirigé (cohérent avec les exemples vus en cours)
- `relation` : type sémantique du lien → utilisé pour filtrer dans Gephi

Remarques sur les relations :

personne-personne
Impossible à extraire depuis le champ nom (chaque nom contient une seule personne).
Les noms de personnes dans le corps (Zacos, Laurent…) sont des chercheurs modernes, pas des Byzantins → bruit trop important, donc pour l’instant non inclus.

personne-religion
Très pertinent :

Justinien Ier → Victoire (avers du sceau)

Nicétas → saint Michel (mention dans le corps)
Cette relation permet d’analyser les préférences des types d’officiers pour certaines images religieuses, mais dépend du traitement du corps du texte.

Person-Symbol
Exemples : christogramme, croix patriarcale.
Ces symboles peuvent être regroupés dans le type religion → pas besoin d’un type Symbol distinct.

Nouvelle version simplifiée :
### edges.csv
| Source | Target | Weight | Type | relation |
|---|---|---|---|---|
| 1 | 2 | 1 | Non dirigé | personne_titre |
| 1 | 3 | 1 | Non dirigé | personne_lieu |
| 4 | 1 | 1 | Non dirigé | personne_religion |

Nœuds (nodes) :

Figure 1 (Harry Potter) : Id et Label identiques (texte), countTerms = fréquence.

Figure 2 (gare) : Id = code OCE, Label = nom géographique, coordonnées, Modularity Class.

Figure 3 (aéroport) : Id = numérique, Label = code aéroport, city, country, coordonnées, Modularity Class.

### nodes.csv
| Id | Label | Type | countTerms | date_debut | date_fin | section |
|---|---|---|---|---|---|---|
| 1 | Justinien Ier | personne | 1 | 527 | 565 | Sceaux impériaux |
| 2 | patrice | titre | 3 | — | — | — |
| 3 | Antioche | lieu | 2 | — | — | — |
| 4 | Théotokos | religion | 5 | — | — | — |

- `Id` : identifiant numérique unique
- `Label` : nom affiché dans Gephi
- `Type` : catégorie du nœud → utilisé pour la couleur (personne / titre / lieu / religion / siecle)
- `countTerms` : nombre total d'apparitions → utilisé pour la taille du nœud
- `siecle` : siècle d'activité (uniquement pour les personnes)

Utilisation de la colonne section

section n’est pas un nœud (seulement 6 valeurs → sinon trop de liens concentrés).

Elle sert comme attribut des nœuds personne → permet de colorer ou filtrer dans Gephi.
Exemple :

Bleu = Sceaux impériaux

Vert = Administration provinciale

Orange = L’Église

Ce n’est pas une relation, mais un filtre visuel :

Ex. : le titre stratège est connecté à plusieurs personnes → les couleurs montrent immédiatement à quels secteurs administratifs ces personnes appartiennent.

Transformer section en nœud créerait un super-nœud et masquerait la structure réelle → mieux vaut rester attribut.

Utilisation de siecle

Dans les 43 entrées, 28 concernent le XIᵉ siècle → plus de 65 %.

Créer un nœud siecle → super-nœud qui masquerait les autres relations.

Les périodes approximatives comme « milieu du XIᵉ siècle » peuvent être converties en intervalle numérique, ex. : 1050-1075.

Dans Gephi, cette colonne permet d’appliquer un dégradé de couleurs : couleurs froides = période ancienne, couleurs chaudes = période tardive, sans conflit avec les couleurs de section.



Informations converties sur les dates :

527‑565 → 7ᵉ siècle

668‑685 → 7ᵉ siècle

11ᵉ siècle (troisième trimestre)

9ᵉ siècle (dernier quart)

XIᵉ siècle moyen

11ᵉ siècle (troisième trimestre)

Xe siècle (seconde moitié)

9ᵉ siècle (dernier tiers)

11ᵉ siècle

11ᵉ siècle (moyen)

Fin Xe – début XIᵉ siècle

11ᵉ siècle (≈ 1057‑1080)

VIIIᵉ siècle (début)

11ᵉ siècle (moyen)

VIIIᵉ siècle

11ᵉ siècle (tiers moyen)

11ᵉ siècle (1050‑1070)

11ᵉ siècle (moyen)

11ᵉ siècle (moyen)

1060‑1080

11ᵉ siècle (dernier tiers)

1060‑1080

11ᵉ siècle (moyen)

11ᵉ siècle (moyen)

11ᵉ siècle (premier quart)

VIᵉ siècle

XIIᵉ siècle (début)

XIIᵉ siècle (début)

Fin XIᵉ – début XIIᵉ siècle (20 premières années)

7ᵉ siècle (seconde moitié)

Début XIᵉ siècle (premières décennies)

1070s (XIᵉ siècle)

Fin XIᵉ – début XIIᵉ siècle

7ᵉ siècle

XIᵉ siècle (première moitié, après 1028‑1029)

7ᵉ siècle (dernier tiers)

XIᵉ siècle (moyen)

11ᵉ siècle (tiers moyen)

11ᵉ siècle

11ᵉ siècle (seconde moitié)

XI‑XIIᵉ siècle

Fin XIᵉ – début XIIᵉ siècle

##Avant de générer ces deux fichiers CSV, il faut d’abord réfléchir à ce que nous voulons observer dans Gephi Ensuite, une fois les fichiers importés et la visualisation créée, nous pourrons adapter notre analyse selon les résultats, le contexte bibliographique et les observations issues de Gephi. Si le volume de données est insuffisant, nous pourrons alors formuler des hypothèses.

Points d’observation

1.Nom de la personne ↔ Titre ↔ Lieu

Si un même nom est connecté à plusieurs lieux et titres, dans Gephi cela se traduira par une structure rayonnante, révélant directement la carrière ou la mobilité géographique de cet officier.

✅ Observable et direct.

2.Titres caractéristiques par dynastie

Certains nœuds titre ne sont-ils connectés qu’à des nœuds dynastie spécifiques ?

Cela peut refléter les réformes de l’administration byzantine.

⚠️ Observation possible, mais dans nos 43 entrées, le XIᵉ siècle domine largement → les autres siècles sont trop peu représentés pour tirer des conclusions robustes.

3.Évolution des éléments religieux

Observer la relation entre éléments religieux et dynastie.

Exemple : le nœud « Vierge » a-t-il plus de connexions aux dynasties des IX‑XIᵉ siècles ?

⚠️ Cela nécessite que le corps du texte soit traité, et reste limité par le volume de données.


4.Préférence des officiels pour certains types d’images religieuses

Permet d’analyser quels types d’officiers favorisent quelles images religieuses.

Dépend du traitement du corps du texte.

## Génération du fichier noued.csv

À utiliser pour le réseau :

nom → personne, titre, lieu

date → dynastie / siècle

corps → éléments religieux (Théotokos, saint Georges…)

À ne pas utiliser :

des → état de conservation (sans rapport avec le réseau sémantique)

inv → numéro d’inventaire (sans rapport avec le réseau)

dia → diamètre (sans rapport avec le réseau)

In [40]:
Concernant la complexité du corps du texte, il y a principalement trois problèmes :

Beaucoup de bruit :

Le corps contient des notes de bas de page, des citations académiques (Zacos, Schlumberger…), ainsi que des lignes de transcription grecque.

Identification des éléments religieux :

Il faut disposer d’une liste de mots-clés (Théotokos, Victoire, saint Michel, christogramme, croix…) pour filtrer correctement.

Variantes d’un même élément :

Par exemple, « Théotokos Blachernitissa » et « Théotokos » désignent le même type de nœud et doivent être normalisés.

### nodes.csv
| Id | Label | Type | countTerms | date_debut | date_fin | section |
|---|---|---|---|---|---|---|
| 1 | Justinien Ier | personne | 1 | 527 | 565 | Sceaux impériaux |
| 2 | patrice | titre | 3 | — | — | — |
| 3 | Antioche | lieu | 2 | — | — | — |
| 4 | Théotokos | religion | 5 | — | — | — |

countTerms correspond au nombre total d’apparitions d’un nœud dans tous les sceaux.

Exemple :

stratège apparaît dans le nom de plusieurs sceaux → countTerms = 5

Antioche n’apparaît que dans deux sceaux → countTerms = 2

nom_str = re.sub(r'\s*\(\?\)', '', nom_str).strip()

Cela supprime les « (?) » des noms, par exemple :

Grègoras (?) → Grègoras

Nicétas (?) Sarônitès → Nicétas Sarônitès

💡 Cette opération permet de nettoyer les noms pour que le label du nœud (Label) ne contienne pas de symboles parasites comme (?).

In [46]:
from google.colab import files
uploaded = files.upload()
CSV_FILE = list(uploaded.keys())[0]
print(f'Fichier : {CSV_FILE}')

Saving 【最终版】sceaux_parsed.csv to 【最终版】sceaux_parsed.csv
Fichier : 【最终版】sceaux_parsed.csv


In [47]:
import pandas as pd
import re
from collections import defaultdict

df = pd.read_csv(CSV_FILE, encoding='utf-8-sig', dtype=str).fillna('')
print(f'{len(df)} entrees chargees')

TITRES = [
    'patrice', 'protospathaire', 'spatharocandidat', 'vestarque', 'vestes',
    'protovestès', 'anthypatos', 'magistre', 'proedre', 'protoproedre',
    'sebaste', 'protonobilissime', 'kouropalate', 'kouropalatissa',
    'stratege', 'stratelates', 'stratopedararque', 'domestique',
    'logothete', 'asekretis', 'chartulaire', 'ostiaire', 'pincerne',
    'xenodoque', 'topoteretes', 'hypatos', 'protokentarque',
    'illoustrios', 'comes', 'duc', 'prefet', 'juge', 'eveque',
    'archeveque', 'syncelle', 'moine', 'econome', 'kathegetes',
    'grand econome', 'grand drongaire', 'basileis des Romains',
    'vestarque', 'vestès', 'proèdre', 'protoproèdre', 'sébaste',
    'stratège', 'stratèlatès', 'stratopédarque', 'logothète',
    'asèkrètis', 'topotèrètès', 'préfet', 'évêque', 'archevêque',
    'économe', 'kathègètès', 'grand économe', 'komès', 'protovestès'
]

LIEUX = [
    'Antioche', 'Dyrrachion', 'Séleucie', 'Opsikion', 'Arithmos',
    'Velum', 'Péloponnèse', 'Hellade', 'Thracésiens', 'Thracèsiens',
    'Bucellaires', 'Excubites', 'Samos', 'Parnassos', 'Éphèse',
    'Sébastè', 'Néa', 'îles', "At'akh", 'Pikridiou'
]

DATE_MAP = {
    '527-565': (527, 565),
    'viie siècle (668-685)': (668, 685),
    'xie siècle (troisième quart)': (1050, 1075),
    'ixe siècle (dernier quart)': (875, 900),
    'milieu du xie siècle': (1025, 1075),
    'xe siècle (deuxième moitié)': (950, 1000),
    'ixe siècle (dernier tiers)': (867, 900),
    'xie siècle': (1000, 1100),
    'xie siècle (milieu)': (1025, 1075),
    'xe (fin)-xie siècle (début)': (980, 1020),
    'xie siècle (env. 1057-1080)': (1057, 1080),
    'viiie siècle (début)': (700, 733),
    'viiie siècle': (700, 800),
    'xie siècle (tiers central)': (1033, 1067),
    'xie siècle (1050-1070)': (1050, 1070),
    '1060-1080': (1060, 1080),
    'dernier tiers du xie siècle': (1067, 1100),
    'xie siècle (premier quart)': (1000, 1025),
    'vie siècle': (500, 600),
    'xiie siècle (début)': (1100, 1133),
    'xie (fin)-xiie (deux premières décennies)': (1080, 1120),
    'viie siècle (seconde moitié)': (650, 700),
    'xie siècle (premières décennies)': (1000, 1040),
    'xie siècle (années 1070)': (1070, 1080),
    'xie siècle (fin)-xiie siècle (début)': (1080, 1110),
    'viie siècle': (600, 700),
    'xie siècle (1re moitié, après 1028-1029)': (1028, 1050),
    'viie siècle (dernier tiers)': (667, 700),
    'xie siècle (seconde moitié)': (1050, 1100),
    'xie-xiie siècle': (1080, 1120),
    'xie siècle (fin) – xiie siècle (début)': (1080, 1110),
    'xie siècle (deuxième moitié)': (1050, 1100)
}

print('Dictionnaires OK')

43 entrees chargees
Dictionnaires OK


In [48]:
# 之前的
# def extraire_nom(nom_str):
#     nom_str = re.sub(r'\s*\(\?\)', '', nom_str).strip()
#     titres_trouves = []
#     lieux_trouves = []
#     for t in sorted(set(TITRES), key=len, reverse=True):
#         if re.search(rf'\b{re.escape(t)}\b', nom_str, re.IGNORECASE):
#             titres_trouves.append(t)
#     for l in sorted(LIEUX, key=len, reverse=True):
#         if re.search(rf'\b{re.escape(l)}\b', nom_str, re.IGNORECASE):
#             lieux_trouves.append(l)
#     personne = nom_str.split(',')[0].strip()
#     return personne, titres_trouves, lieux_trouves

# for _, row in df.iterrows():
#     p, t, l = extraire_nom(row['nom'])
#     print(f"[{row['numero']}] personne={p} | titres={t} | lieux={l}")

[1] personne=Justinien Ier | titres=[] | lieux=[]
[2] personne=Constantin IV | titres=['basileis des Romains'] | lieux=[]
[3] personne=Georges | titres=['grand économe', 'économe', 'patrice', 'juge'] | lieux=['Velum']
[4] personne=Grègoras | titres=['spatharocandidat', 'topotèrètès'] | lieux=['Arithmos']
[5] personne=Michel | titres=['vestarque', 'économe'] | lieux=['Néa']
[6] personne=Nicétas Sarônitès | titres=['protovestès', 'domestique'] | lieux=['Excubites']
[7] personne=Nicétas | titres=['pincerne', 'ostiaire'] | lieux=[]
[8] personne=Théodore | titres=['protospathaire', 'anthypatos', 'logothète', 'patrice'] | lieux=[]
[9] personne=David Tzipourélès | titres=['protospathaire', 'stratège'] | lieux=[]
[10] personne=Étienne | titres=['vestarque', 'duc'] | lieux=['Antioche']
[11] personne=Jean Radènos | titres=['protospathaire', 'anthypatos', 'stratège', 'patrice'] | lieux=[]
[12] personne=Jean | titres=['protoproèdre', 'duc'] | lieux=['Antioche']
[13] personne=Justinien | titres=['t

In [60]:
def extraire_nom(nom_str):
    nom_str = re.sub(r'\s*\(\?\)', '', nom_str).strip()
    titres_trouves = []
    lieux_trouves = []
    for t in sorted(set(TITRES), key=len, reverse=True):
        if re.search(rf'\b{re.escape(t)}\b', nom_str, re.IGNORECASE):
            titres_trouves.append(t)
    for l in sorted(LIEUX, key=len, reverse=True):
        if re.search(rf'\b{re.escape(l)}\b', nom_str, re.IGNORECASE):
            lieux_trouves.append(l)
    personne = nom_str.split(',')[0].strip()
    MOTS_NON_NOM = ['kathègètès', 'kathegetes']
    if any(m in personne.lower() for m in MOTS_NON_NOM):
        personne = None
    return personne, titres_trouves, lieux_trouves

for _, row in df.iterrows():
    p, t, l = extraire_nom(row['nom'])
    print(f"[{row['numero']}] personne={p} | titres={t} | lieux={l}")

[1] personne=Justinien Ier | titres=[] | lieux=[]
[2] personne=Constantin IV | titres=['basileis des Romains'] | lieux=[]
[3] personne=Georges | titres=['grand économe', 'économe', 'patrice', 'juge'] | lieux=['Velum']
[4] personne=Grègoras | titres=['spatharocandidat', 'topotèrètès'] | lieux=['Arithmos']
[5] personne=Michel | titres=['vestarque', 'économe'] | lieux=['Néa']
[6] personne=Nicétas Sarônitès | titres=['protovestès', 'domestique'] | lieux=['Excubites']
[7] personne=Nicétas | titres=['pincerne', 'ostiaire'] | lieux=[]
[8] personne=Théodore | titres=['protospathaire', 'anthypatos', 'logothète', 'patrice'] | lieux=[]
[9] personne=David Tzipourélès | titres=['protospathaire', 'stratège'] | lieux=[]
[10] personne=Étienne | titres=['vestarque', 'duc'] | lieux=['Antioche']
[11] personne=Jean Radènos | titres=['protospathaire', 'anthypatos', 'stratège', 'patrice'] | lieux=[]
[12] personne=Jean | titres=['protoproèdre', 'duc'] | lieux=['Antioche']
[13] personne=Justinien | titres=['t

In [66]:
nodes = {}
edge_counts = defaultdict(int)
node_id_counter = [1]

SYNONYMES_LIEUX = {
    'Thracèsiens': 'Thracésiens',
}

# Labels personnalisés pour les homonymes
LABEL_HOMONYMES = {
    '7':   'Nicétas, ostiaire',
    '18':  'Nicétas, juge',
    '19':  'Nicétas, protospathaire',
    '5':   'Michel, vestarque',
    '17':  'Michel, magistre',
    '8':   'Théodore, logothète',
    '24':  'Théodore, préfet',
    '12':  'Jean, duc',
    '33':  'Jean, évêque',
    '14':  'Léon, vestès',
    '15':  'Léon, protospathaire',
    '20':    'Oursel de Bailleul (20)',
    '20bis': 'Oursel de Bailleul (20bis)',
}

def normaliser_lieu(l):
    return SYNONYMES_LIEUX.get(l, l)

def get_label_personne(personne, numero):
    return LABEL_HOMONYMES.get(str(numero), personne)

def get_or_create_node(label, ntype, date_debut='', date_fin='', section=''):
    if label not in nodes:
        nodes[label] = {
            'Id': node_id_counter[0],
            'Label': label,
            'Type': ntype,
            'countTerms': 0,
            'date_debut': date_debut,
            'date_fin': date_fin,
            'section': section
        }
        node_id_counter[0] += 1
    nodes[label]['countTerms'] += 1
    return nodes[label]['Id']

for _, row in df.iterrows():
    date = row['date'].strip().lower()
    section = row['section'].strip()
    debut, fin = DATE_MAP.get(date, ('', ''))
    personne, titres, lieux = extraire_nom(row['nom'])
    if not personne:
        continue

    # Label avec distinction pour les homonymes
    label_personne = get_label_personne(personne, row['numero'])
    cle_personne = f"{personne}__{row['numero']}"

    if cle_personne not in nodes:
        nodes[cle_personne] = {
            'Id': node_id_counter[0],
            'Label': label_personne,
            'Type': 'personne',
            'countTerms': 0,
            'date_debut': str(debut),
            'date_fin': str(fin),
            'section': section
        }
        node_id_counter[0] += 1
    nodes[cle_personne]['countTerms'] += 1
    pid = nodes[cle_personne]['Id']

    for t in titres:
        tid = get_or_create_node(t, 'titre')
        edge_counts[(pid, tid, 'personne_titre')] += 1

    for l in lieux:
        l = normaliser_lieu(l)
        lid = get_or_create_node(l, 'lieu')
        edge_counts[(pid, lid, 'personne_lieu')] += 1

nodes_df = pd.DataFrame(list(nodes.values()))
display(nodes_df)
print(f'Noeuds : {len(nodes)}')
print(f'Aretes : {len(edge_counts)}')

,Id,Label,Type,countTerms,date_debut,date_fin,section
0,1,Justinien Ier,personne,1,527,565,Sceaux impériaux
1,2,Constantin IV,personne,1,668,685,Sceaux impériaux
2,3,basileis des Romains,titre,1,,,
3,4,Georges,personne,1,1050,1075,L'administration centrale
4,5,grand économe,titre,1,,,
...,...,...,...,...,...,...,...
91,92,Sébastè,lieu,1,,,
92,93,Théodose,personne,1,1000,1100,Les dignitaires
93,94,Constantin Babôskomètès,personne,1,1080,1120,Sceaux patronymiques
94,95,Basile Pépagoménos,personne,1,1080,1110,Sceaux patronymiques


Noeuds : 96
Aretes : 87


In [67]:


nodes_df = pd.DataFrame(list(nodes.values()))
print('=== Personnes ===')
display(nodes_df[nodes_df['Type']=='personne'][['Id','Label','date_debut','date_fin','section','countTerms']])
print('\n=== Titres ===')
display(nodes_df[nodes_df['Type']=='titre'][['Id','Label','countTerms']].sort_values('countTerms', ascending=False))
print('\n=== Lieux ===')
display(nodes_df[nodes_df['Type']=='lieu'][['Id','Label','countTerms']].sort_values('countTerms', ascending=False))



=== Personnes ===


,Id,Label,date_debut,date_fin,section,countTerms
0,1,Justinien Ier,527,565,Sceaux impériaux,1
1,2,Constantin IV,668,685,Sceaux impériaux,1
3,4,Georges,1050,1075,L'administration centrale,1
9,10,Grègoras,875,900,L'administration centrale,1
13,14,"Michel, vestarque",1025,1075,L'administration centrale,1
16,17,Nicétas Sarônitès,1050,1075,L'administration centrale,1
20,21,"Nicétas, ostiaire",950,1000,L'administration centrale,1
23,24,"Théodore, logothète",867,900,L'administration centrale,1
27,28,David Tzipourélès,1000,1100,Administration provinciale,1
29,30,Étienne,1025,1075,Administration provinciale,1



=== Titres ===


,Id,Label,countTerms
24,25,protospathaire,8
28,29,stratège,5
7,8,juge,4
6,7,patrice,4
14,15,vestarque,3
80,81,évêque,3
11,12,topotèrètès,3
5,6,économe,2
68,69,sébaste,2
56,57,proèdre,2



=== Lieux ===


,Id,Label,countTerms
31,32,Antioche,2
52,53,Thracésiens,2
37,38,Opsikion,2
8,9,Velum,1
15,16,Néa,1
19,20,Excubites,1
40,41,Bucellaires,1
42,43,Dyrrachion,1
12,13,Arithmos,1
46,47,Péloponnèse,1


Première version du CSV nodes :

Problèmes relevés :

Nœuds personne en double

Nicétas apparaît deux fois (Id 17 et 21) :

Nicétas Sarônitès (entrée 6)

Nicétas (entrées 7 et 18‑19)

Ces trois Nicétas sont différentes personnes, mais lors de l’extraction, seul le prénom avant la virgule a été pris → fusion en un seul nœud Nicétas avec countTerms = 3.

Solution : conserver le nom complet pour les distinguer.

Nœuds lieu en double

Thracésiens (Id 50) et Thracèsiens (Id 51) désignent le même lieu, mais orthographe différente → fusionner.

Section incorrecte pour certaines personnes

Exemple : Michel (Id 14) est indiqué dans la section L'administration centrale, alors qu’il a deux entrées (5 et 17) appartenant à des sections différentes → la fusion a fait perdre l’information de section.

Nom de poste pris comme personne

Kathègètès du monastère de ta Pikridiou (entrée 37) est un titre/position, pas un nom → ne doit pas devenir un nœud personne.

Jean apparaît deux fois (Id 34)

Jean Radènos (entrée 11) et Jean (entrée 12) sont différentes personnes, mais l’extraction a gardé seulement Jean → fusion.

Id 33 = Jean Radènos → correct.

Deuxième version du CSV nodes :

Amélioration générale, mais problèmes restants :

Nicétas en double (Id 52 et 54)

Date et section identiques (1025‑1075, Administration provinciale) → entrées 18 et 19

Ces deux Nicétas sont différentes personnes avec le même prénom

Question : fusionner ou conserver séparément en utilisant le nom complet (mais le PDF ne donne pas de nom complet).

Oursel (Roussel) de Bailleul en double (Id 55 et 58)

Même personne (entrées 20 et 20bis), dates légèrement différentes

Conserver les deux nœuds est correct, mais Label identique → confusion dans Gephi

Suggestion :

Oursel de Bailleul (20)

Oursel de Bailleul (20bis)

Duplications normales pour des homonymes

Michel 14 vs 49 ✅

Théodore 24 vs 65 ✅

Jean 34 vs 87 ✅

Léon 39 vs 42 ✅

Nicétas 21 vs 52/54 ✅

Pour ces homonymes : recherche AI pour compléter noms et prénoms impossible, car ce sont de petits personnages byzantins semi-anonymes.

Le PDF est la source la plus fiable : le corps contient des discussions sur l’identité des personnages.

Souvent, le corps donne juste des références parallèles (DOSeals, Laurent…) → pas de nom complet.

C’est un problème classique en sigillographie byzantine : sceaux existants, personnes perdues dans l’histoire.

Troisième version du CSV nodes :

Presque tous les problèmes réglés, sauf un :

Oursel (Roussel) de Bailleul encore en double (Id 55 et 58) → entrées 20 et 20bis

Même personne, mais Label identique → superposition dans Gephi

Solution : ajouter le numéro d’entrée au Label pour distinguer :

Oursel de Bailleul (20)

Oursel de Bailleul (20bis)

In [ ]:
Toutes les informations sauf celles liées à la religion ont été traitées.
Nous allons maintenant commencer à traiter les éléments religieux (religion).

In [ ]:
Nous voulons les noms des saints et les noms de symboles présents dans le corps du texte (corps), qui constitueront les nœuds de type religion.

D’après ce que vous avez dit, il n’est pas nécessaire d’avoir une colonne Symbol distincte, mais il faudra quand même réfléchir pendant le traitement pour décider si un jour un type séparé serait pertinent.

Iconographie des saints :

Fonctions :

Protection (armée, territoire, personne)

Montrer l’autorité ou le statut (empereur, noblesse, Église)

Saints fréquemment mentionnés :

Vierge Marie (Théotokos)

Saint Nicolas (Saint Nicolas)

Saint Démétrius (Saint Démétrius)

Saint Georges (Saint Georges)

Saints Pierre et Paul → utilisés spécifiquement à Antioche


In [ ]:
Concernant la colonne symbol, mon avis est qu’elle n’est pas nécessaire.

Raisons :

Du point de vue de Gephi, l’objectif est d’étudier « quel type d’officiel préfère quel type d’image religieuse ».

Que ce soit un saint ou un symbole (christogramme, croix patriarcale…), dans le réseau, ils remplissent le même rôle : ce sont les associations religieuses d’un nœud personne.

Créer un type séparé symbol fragmenterait la relation et compliquerait l’analyse : deux types de nœuds remplissant la même fonction apparaîtraient dans le graphique.

Solution :

Tout regrouper sous religion comme type de nœud.

Le Label distinguera :

Saints : Théotokos, saint Michel, saint Georges…

Symboles : croix patriarcale, christogramme, Victoire…

Si nécessaire, ajouter une colonne sous_type (saint / symbole) uniquement pour les nœuds religion, pour filtrer ou colorer dans Gephi, mais cela n’affecte pas la structure des liens.

Organisation finale des attributs :
Colonne	Utilisation	Remarques
Type	personne / titre / lieu / religion	utilisé pour le filtrage
sous_type	saint / symbole	uniquement pour religion, différenciation interne
section	6 catégories administratives	uniquement pour personne, filtrage par catégorie
relation	type du lien	filtrage par type de lien (personne_titre, personne_religion…)
Dans Gephi, cela permet :

Type (Nœud) → filtrer personne / titre / lieu / religion

sous_type (Nœud) → voir uniquement les saints ou uniquement les symboles

section (Nœud) → voir uniquement les personnes d’une certaine catégorie administrative

relation (Lien) → filtrer par type de lien (personne_titre, personne_religion, etc.)

Cela garantit que l’analyse reste simple, cohérente et fidèle aux données du PDF.

In [ ]:
Tout d’abord, observons les informations de la colonne corps :

Fusionner les doublons / synonymes :

Vierge et Théotokos → même icône, unifier sous Théotokos

saint Dèmètrios et saint Démétrios → unifier sous saint Démétrios

saint georges → uniformiser la casse → saint Georges

Bruit dans l’entrée 20 :

Victoire apparaît dans l’entrée 20, mais il s’agit de discussions académiques sur d’autres sceaux, pas de l’image propre à l’entrée 20 → exclure manuellement.

« croix » trop générique :

croix et croix patriarcale apparaissent souvent ensemble, car la description de croix patriarcale contient toujours le mot croix.

Recommandation : ne garder que croix patriarcale, supprimer les mentions isolées de croix.

In [74]:
# ── Corps : extraction des éléments religieux ────────────────────────────────

SAINTS_MAP = {
    'saint georges'     : 'saint Georges',
    'saint michel'      : 'saint Michel',
    'saint théodore'    : 'saint Théodore',
    'saint nicolas'     : 'saint Nicolas',
    'saint démétrios'   : 'saint Démétrios',
    'saint dèmètrios'   : 'saint Démétrios',
    'théotokos'         : 'Théotokos',
    'vierge'            : 'Théotokos',       # fusionné
    'christ'            : 'Christ',
    'victoire'          : 'Victoire',
    'saint militaire'   : 'saint militaire',
}

SYMBOLES_MAP = {
    'croix patriarcale'      : 'croix patriarcale',
    'christogramme'          : 'christogramme',
    'monogramme invocatif'   : 'monogramme invocatif',
    'monogramme'             : 'monogramme',
    'labarum'                : 'labarum',
    # 'croix' seul → ignoré (trop générique)
}

# Exclusions manuelles : (numero, element) à ne pas créer
EXCLUSIONS = {
    ('20', 'Victoire'),   # mentionné dans discussion académique, pas l'image du sceau
}

religion_nodes = {}   # label → {sous_type, countTerms}
religion_edges = []   # (personne_id, label)

for _, row in df.iterrows():
    """
    corps = row['corps'].lower()
    numero = str(row['numero'])
    """

    corps_temp = row['corps'].lower()
    numero = str(row['numero'])


    elements = []

    # Chercher saints (plus long en premier pour éviter les sous-chaînes)
    ## Saints : après match, effacer du texte pour éviter les sous-chaînes
    """
    for key in sorted(SAINTS_MAP, key=len, reverse=True):
        if key in corps:
            elements.append(('saint', SAINTS_MAP[key]))
    """

    for key in sorted(SAINTS_MAP, key=len, reverse=True):
        if key in corps_temp:
            elements.append(('saint', SAINTS_MAP[key]))
            corps_temp = corps_temp.replace(key, '')

    # Chercher symboles
    ## Symboles : idem, monogramme invocatif matché en premier, puis monogramme ne matche plus
    """
    for key in sorted(SYMBOLES_MAP, key=len, reverse=True):
        if key in corps:
            elements.append(('symbole', SYMBOLES_MAP[key]))
     """
    for key in sorted(SYMBOLES_MAP, key=len, reverse=True):
        if key in corps_temp:
            elements.append(('symbole', SYMBOLES_MAP[key]))
            corps_temp = corps_temp.replace(key, '')

    # Dédoublonner(au cas où deux entrées pointent vers le même label normalisé)
    elements = list(dict.fromkeys(elements))

    # Appliquer exclusions
    elements = [(st, lb) for st, lb in elements if (numero, lb) not in EXCLUSIONS]

    for sous_type, label in elements:
        if label not in religion_nodes:
            religion_nodes[label] = {'sous_type': sous_type, 'countTerms': 0}
        religion_nodes[label]['countTerms'] += 1
        religion_edges.append((numero, label))

print(f"Éléments religion : {len(religion_nodes)}")
for label, info in sorted(religion_nodes.items(), key=lambda x: -x[1]['countTerms']):
    print(f"  {info['countTerms']}x  [{info['sous_type']}]  {label}")

Éléments religion : 13
  7x  [saint]  Christ
  7x  [saint]  Théotokos
  7x  [saint]  saint Georges
  5x  [saint]  saint Théodore
  4x  [symbole]  croix patriarcale
  4x  [saint]  saint Michel
  4x  [saint]  saint Nicolas
  3x  [symbole]  monogramme
  2x  [saint]  saint Démétrios
  2x  [symbole]  monogramme invocatif
  1x  [saint]  Victoire
  1x  [symbole]  labarum
  1x  [saint]  saint militaire


14 éléments religieux ont été extraits de manière claire. Deux points à confirmer :

Victoire (1 occurrence)

Apparaît dans l’entrée 1 (Justinien Ier).

Il s’agit de l’image réelle sur le sceau (déesse Victoire à l’avers), pas d’une discussion académique.

Nous avons précédemment exclu Victoire dans l’entrée 20, mais la conserver dans l’entrée 1 est correct.

✅ Confirmé

saint militaire (1 occurrence)

Apparaît dans l’entrée 34 (Nicéphore, moine).

L’auteur écrit : « buste de saint militaire dont… renvoie a priori à saint Georges », donc l’identité exacte n’est pas confirmée.

Il faut conserver saint militaire et ne pas le fusionner avec saint Georges.

✅ Confirmé

In [75]:
# ── Construction des nœuds et arêtes religion ────────────────────────────────
"""
# Récupérer le mapping numero → personne_id depuis nodes
numero_to_pid = {}
for _, row in df.iterrows():
    cle = f"{row['nom'].split(',')[0].strip().replace('(?)', '').strip()}__{row['numero']}"
    # Chercher dans nodes par clé
    if cle in nodes:
        numero_to_pid[str(row['numero'])] = nodes[cle]['Id']
"""

# Récupérer le mapping numero → personne_id depuis nodes
numero_to_pid = {}
for cle, node in nodes.items():
    # La clé est de la forme "NOM__NUMERO"
    if '__' in cle:
        numero = cle.split('__')[-1]
        numero_to_pid[str(numero)] = node['Id']

print("Mapping numero → pid :")
for k, v in sorted(numero_to_pid.items()):
    print(f"  {k} → {v}")
# Ainsi, en extrayant directement "numero" depuis les clés du dictionnaire nodes,
# au lieu de reparcourir le champ "nom", le mappage devient plus stable.

# L'avertissement pour numero=37 disparaîtra car cette entrée n'a pas de nœud personne.
# On pourrait ajouter une ligne pour la filtrer,
# mais dans le code, il y a déjà "if pid is None: continue" → l'avertissement n'affecte pas le résultat.

# Après avoir remplacé la partie mapping, les liens pour les entrées 6 et 11
# peuvent être correctement établis.

# Construire religion_df
max_id = max(n['Id'] for n in nodes.values())
religion_rows = []
religion_label_to_id = {}

for i, (label, info) in enumerate(religion_nodes.items()):
    nid = max_id + i + 1
    religion_label_to_id[label] = nid
    religion_rows.append({
        'Id': nid,
        'Label': label,
        'Type': 'religion',
        'countTerms': info['countTerms'],
        'sous_type': info['sous_type'],
        'date_debut': '',
        'date_fin': '',
        'section': ''
    })

religion_df = pd.DataFrame(religion_rows)

# Construire religion_edges_df
religion_edge_counts = defaultdict(int)
for numero, label in religion_edges:
    pid = numero_to_pid.get(numero)
    if pid is None:
        print(f"  ⚠ pas de personne trouvée pour numero={numero}")
        continue
    rid = religion_label_to_id[label]
    religion_edge_counts[(pid, rid, 'personne_religion')] += 1

religion_edges_df = pd.DataFrame([
    {'Source': s, 'Target': t, 'Weight': w, 'Type': 'Non dirigé', 'relation': r}
    for (s, t, r), w in religion_edge_counts.items()
])

print(f"Nœuds religion : {len(religion_df)}")
print(f"Arêtes religion : {len(religion_edges_df)}")
display(religion_df)

Mapping numero → pid :
  1 → 1
  10 → 30
  11 → 33
  12 → 34
  13 → 36
  14 → 39
  15 → 42
  16 → 44
  17 → 49
  18 → 52
  19 → 54
  2 → 2
  20 → 55
  20bis → 58
  20ter → 59
  21 → 61
  22 → 62
  23 → 64
  24 → 65
  25 → 68
  25bis → 70
  26 → 72
  27 → 73
  28 → 76
  29 → 77
  3 → 4
  30 → 79
  31 → 80
  32 → 83
  33 → 87
  34 → 89
  35 → 91
  36 → 93
  38 → 94
  39 → 95
  4 → 10
  40 → 96
  5 → 14
  6 → 17
  7 → 21
  8 → 24
  9 → 28
  ⚠ pas de personne trouvée pour numero=37
Nœuds religion : 13
Arêtes religion : 47


,Id,Label,Type,countTerms,sous_type,date_debut,date_fin,section
0,97,Victoire,religion,1,saint,,,
1,98,Christ,religion,7,saint,,,
2,99,Théotokos,religion,7,saint,,,
3,100,saint Georges,religion,7,saint,,,
4,101,croix patriarcale,religion,4,symbole,,,
5,102,saint Michel,religion,4,saint,,,
6,103,saint Théodore,religion,5,saint,,,
7,104,saint Démétrios,religion,2,saint,,,
8,105,monogramme invocatif,religion,2,symbole,,,
9,106,monogramme,religion,3,symbole,,,



Première version finale du CSV nodes

Les nœuds religion sont corrects, mais quatre avertissements doivent être corrigés :

numero=6 → Nicétas Sarônitès (nom complet, clé non trouvée)

numero=11 → Jean Radènos (idem)

numero=37 → Kathègètès (pas de nœud personne, normal)

Problème : ces personnes ont un nom complet, mais lors de la construction de la clé, seule la partie avant la virgule était prise. Or, le champ nom peut contenir des espaces ou des (?), ce qui provoque l’échec de la correspondance.

Exemple : extraction depuis le champ nom :

Nicétas (?) Sarônitès, protovestès... → supprimer (?) → Nicétas Sarônitès → clé = Nicétas Sarônitès__6

Jean (?) Radènos, anthypatos... → supprimer (?) → Jean Radènos → clé = Jean Radènos__11

C’est la clé utilisée lors de la création du nœud (cellule 4) → correcte.
Mais dans la cellule 4b, j’ai reconstruit le mapping avec une autre logique qui reparsait le champ nom → incohérence → échec de correspondance.

Cause principale : la cellule 4b reparse inutilement le nom, au lieu de lire directement le dictionnaire nodes.

Solution : modifier la construction du mapping dans 4b pour lire directement les clés du dictionnaire nodes.

Deuxième version finale du CSV nodes

Très bon résultat : 110 nœuds, structure complète.

Petit problème :

monogramme (Id 107) et monogramme invocatif (Id 106) en double

monogramme invocatif est un sous-ensemble de monogramme → si présent dans corps, les deux correspondent → le même sceau connecté aux deux nœuds.

Solution :

Dans la logique d’extraction de SYMBOLES_MAP : une fois que monogramme invocatif est trouvé, ne pas matcher monogramme.

Même si les mots-clés sont triés par longueur, il faut une extraction mutuellement exclusive.

Correctif simple : dans la boucle d’extraction (cellule 4b), remplacer dans corps les mots-clés déjà matchés avant de continuer la recherche pour les suivants.


In [76]:
# ── Fusion et export final ────────────────────────────────────────────────────

# Ajouter sous_type aux nodes existants (vide pour personne/titre/lieu)
nodes_df['sous_type'] = ''

# Fusionner nodes
nodes_final = pd.concat([nodes_df, religion_df], ignore_index=True)

# Fusionner edges
edges_final = pd.concat([edges_df, religion_edges_df], ignore_index=True)

print(f"Total nœuds : {len(nodes_final)}  (personne={len(nodes_final[nodes_final['Type']=='personne'])}, titre={len(nodes_final[nodes_final['Type']=='titre'])}, lieu={len(nodes_final[nodes_final['Type']=='lieu'])}, religion={len(nodes_final[nodes_final['Type']=='religion'])})")
print(f"Total arêtes : {len(edges_final)}")

from google.colab import files
nodes_final.to_csv('nodes_final.csv', index=False, encoding='utf-8-sig')
edges_final.to_csv('edges_final.csv', index=False, encoding='utf-8-sig')
files.download('nodes_final.csv')
files.download('edges_final.csv')

Total nœuds : 109  (personne=42, titre=37, lieu=17, religion=13)
Total arêtes : 134


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>